# Import thư viện cần thiết

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

Đọc dữ liệu

In [2]:
train = pd.read_csv('../data/train_preprocessed.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [3]:
test = pd.read_csv('../data/test_preprocessed.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,121.00,324


In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)

tar = all_data['Target'].copy()   # Tách cột target và lưu tạm thời.
all_data.drop(columns='Target', inplace=True) 

# Xây dựng mô hình học máy

## Loại bỏ những thuộc tính dư thừa

### Tạo biến ordinal từ dữ liệu đã được one-hot encode

Các thuộc tính như `epared`, `etecho`, `eviv` và `instlevel` có thể được chuyển về dạng dữ liệu ordinal với quy ước **(bad, regular, good) -> (0, 1, 2)**

In [5]:
def get_numeric(data, status_name):
    status_cols = [s for s in data.columns.tolist() if status_name in s]
    status_df = data[status_cols]
    status_df.columns = list(range(status_df.shape[1]))
    status_numeric = status_df.idxmax(1)
    status_numeric.name = status_name
    data = pd.concat([data, status_numeric], axis=1)
    return data

In [6]:
status_name_list = ['epared', 'etecho', 'eviv', 'instlevel']
for status_name in status_name_list:
    all_data = get_numeric(all_data, status_name)

### Xóa những thuộc tính không cần thiết

Nhóm nhận thấy có những thuộc tính có thể được xác định bằng những thuộc tính khác trong dữ liệu.

* `v18q` có thể được tạo ra từ `v18q1`.
* `mobilephone` có thể được tạo ra từ `qmobilephone`.
* `epared1~3`, `etecho1~3`, `eviv1~3`, `instlevel1~9` do đã được chuyển đổi thành dữ liệu ordinal nên sẽ không dùng đến nữa.

* Các cột được bình phương `SQB*`

In [7]:
# Loại bỏ những cột đã được label encoder
useless_col = ['epared1', 'epared2', 'epared3', 
                'etecho1', 'etecho2', 'etecho3',
                'eviv1', 'eviv2', 'eviv3', 
                'instlevel1', 'instlevel2', 'instlevel3', 'instlevel4', 'instlevel5', 'instlevel6', 'instlevel7', 'instlevel8', 'instlevel9',
                'SQBescolari', 'SQBage', 'agesq', 'SQBhogar_total',
                'SQBedjefe', 'SQBhogar_nin', 'SQBovercrowding',
                'SQBdependency', 'SQBmeaned']
all_data.drop(columns=useless_col, inplace=True)

### Điền dữ liệu thiếu
- Cột `rez_esc`, `v2a1` không có thêm thông tin để giải quyết nên sẽ fill dữ liệu thiếu là 0

In [8]:
all_data['rez_esc'].fillna(0, inplace =True)   
all_data['v2a1'].fillna(0, inplace =True) 

## Tạo các cột mới

### Thêm cột dựa trên tình hình cơ sở vật chất

Nhóm thêm cột `minus` và `bonus` cho điểm trừ, điểm cộng dựa trên các đặc điểm của csvc

Đối với điểm trừ, các hộ gia đình mà không có:
- toilet, điện nước, gas, tường....
- Điểm trừ sẽ là số âm

Đối với điểm cộng, các hộ gia đình có:
- tủ lạnh, máy tính, lượng máy tính bảng, tivi....

In [9]:
all_data['minus'] = -1 * (all_data['sanitario1'] +   # không toilet
                         (all_data['noelec'] == 0) + # không có điện
                         all_data['pisonotiene'] +   # không có sàn
                         all_data['abastaguano'] +   # không có nước
                         (all_data['cielorazo'] == 0)+    # không có trần
                         (all_data['energcocinar1'] )+# không có năng lượng để nấu ăn
                         (all_data['epared'] == 0 )+      # tường bad
                         (all_data['etecho'] == 0 )+      # mái nhà bad
                         (all_data['eviv'] == 0 ))        # sàn nhà bad


all_data['bonus'] = 1 * (all_data['refrig'] +    # tủ lạnh
                      all_data['computer'] +    # máy tính
                      (all_data['v18q1'] > 0) + # có tablet 
                      all_data['television'])   # có tivi


### Tạo các cột mới theo tỉ lệ (phòng, tiền nhà) / người


In [10]:

def extract_features(df):
    df['bedrooms_to_rooms'] = df['bedrooms']/df['rooms']  # tỉ lệ phòng ngủ trong nhà
    df['rent_per_bedroom'] = df['v2a1']/df['bedrooms']    # tiền thuê phòng ngủ
    df['adults_per_bedroom'] = df['hogar_adul']/df['bedrooms'] # bn người lớn/phòng ngủ
    df['child_per_bedroom'] = df['hogar_nin']/df['bedrooms']
    df['male_per_bedroom'] = df['r4h3']/df['bedrooms']
    df['female_per_bedroom'] = df['r4m3']/df['bedrooms']


    df['overcrowding_room_and_bedroom'] = (df['hacdor'] + df['hacapo'])/2  # số phòng quá đông đúc
    df['rent_to_rooms'] = df['v2a1']/df['rooms']          # tiền thuê mỗi phòng
    df['tamhog_to_rooms'] = df['tamhog']/df['rooms']    # bao nhiêu người/phòng
    
    df['r4t3_to_tamhog'] = df['r4t3']/df['tamhog']      # r4t3 - Total persons in the household
    df['r4t3_to_rooms'] = df['r4t3']/df['rooms']        # bao nhiêu người/phòng
    df['v2a1_to_r4t3'] = df['v2a1']/df['r4t3']          # tiền thuê nhà theo đầu người
    df['v2a1_to_r4t3'] = df['v2a1']/(df['r4t3'] - df['r4t1']) # tiền thuê nhà ko tính trẻ em (<12 tuối)
    
    df['hhsize_to_rooms'] = df['hhsize']/df['rooms'] # bao nhiêu người/phòng
    df['rent_per_person'] = df['v2a1']/df['hhsize'] # tiển thuê chia theo hộ gia đình
    # df['rent_per_adult'] = df['v2a1']/df['hogar_adul'] #tiền thuê chia cho adult

    
extract_features(all_data)       

In [11]:
all_data['hogar_adul'][all_data['hogar_adul'] !=0 ].sum()

87021

Sau khi đã rút ra các đặc trưng cần thiết, xóa các cột trên

In [12]:
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a',
                 'mobilephone', 'female']

all_data.drop(needless_cols, axis=1, inplace=True)
all_data.shape

(33413, 127)

### Đến đây ta có thể dùng dữ liệu cho model.

In [13]:
all_data = pd.concat([all_data, tar],axis=1) # thêm cột target 
# Lọc ra tâp train, tập test
train = all_data.loc[all_data["Target"].notnull()]
test = all_data.loc[all_data["Target"].isnull()]

In [14]:
X= train.drop( ['Id', 'idhogar', 'Target'], axis= 1)
y= train['Target']

### Scale dữ liệu
- Đối với cột có giá trị lớn (từ 100-100000), nhóm sẽ dùng Standard Scaler để đưa dữ liệu về phân phối chuẩn
- Đối với các cột còn lại, nhóm sẽ dùng MinMax Scaler để đưa giá trị về khoảng (0,1) 

In [15]:
# lọc ra những cột dùng standard scaler
col_std = ['v2a1', 'edjefe', 'edjefa', 'meaneduc', 'age',
             'rent_to_rooms', 'v2a1_to_r4t3', 
             'rent_per_bedroom',  'rent_to_rooms', 'v2a1_to_r4t3', 'rent_per_person']

# các cột còn lại dùng minmax
col_minmax =  list(set(X.columns) - set(col_std))

### Xử lý mất cân bằng dữ liệu


In [16]:
print('Tỉ lệ các label để train: ',
      train['Target'].value_counts()/train.shape[0])

Tỉ lệ các label để train:  Target
4.0    0.627394
2.0    0.167103
3.0    0.126504
1.0    0.079000
Name: count, dtype: float64


Theo tỉ lệ trên có thể thấy được label 4 gấp 10 lần label 1. Nếu không xử lý có thể làm mô hình dự đoán không hiệu quả.

Cách xử lý: Gán cho mỗi label một trọng số. Trọng số này tỉ lệ nghịch với số lượng label. Dữ liệu nào ít nhãn thì có trọng số cao. 

Trọng số này có tác dụng để `phạt` mô hình. Khi mô hình dự đoán sai nhãn có trọng số cao (số lượng nhãn ít) thì hàm lost lại dữ liệu đó cao.

In [17]:
y_weights = class_weight.compute_sample_weight('balanced', y, indices=None)

print(pd.DataFrame(dict(Target = y, Weight = y_weights)).drop_duplicates().sort_values(by = ["Target"]).reset_index(drop = True))

   Target    Weight
0     1.0  3.164570
1     2.0  1.496086
2     3.0  1.976220
3     4.0  0.398474


In [18]:
# phân chia tập train, test, và weight tương ứng.
X_train,X_test,y_train,y_test, weights_train,weights_test  =train_test_split(X,y,y_weights,test_size=0.2, random_state=1)
X_train.shape

(7645, 125)

### Tạo pipeline để train model

Pipe line gồm : chuẩn hóa dữ liệu ( standard, minmax) -> train model

Ở đây, có tham số `best_param`. Tham số này được chạy từ GridSearch để chọn ra bộ tham số tốt nhất cho mô hình

In [19]:
# Tạo Pipeline preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('std_scaler', StandardScaler(), col_std),  # Chuẩn hóa các cột trong nhóm col_std
        ('minmax_scaler', MinMaxScaler(), col_minmax)  # Chuẩn hóa các cột trong nhóm col_minmax
    ])

# Khởi tạo GradientBoostingClassifier
gradient = GradientBoostingClassifier()

# Tạo pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Chuẩn hóa dữ liệu
    ('classifier', gradient)  # Mô hình phân loại
])

# Siêu tham số đã được chọn từ GS
best_param =  {'classifier__learning_rate': 0.3, 
               'classifier__max_depth': 7, 
               'classifier__n_estimators': 200}


pipeline.set_params(**best_param) # set siêu tham số cho mô hình

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('std_scaler',
                                                  StandardScaler(),
                                                  ['v2a1', 'edjefe', 'edjefa',
                                                   'meaneduc', 'age',
                                                   'rent_to_rooms',
                                                   'v2a1_to_r4t3',
                                                   'rent_per_bedroom',
                                                   'rent_to_rooms',
                                                   'v2a1_to_r4t3',
                                                   'rent_per_person']),
                                                 ('minmax_scaler',
                                                  MinMaxScaler(),
                                                  ['qmobilephone', 'noelec',
                                                   'tipovivi3', 'techootro',
                                                   'energcocinar3',
                                                   'sanitario3', '...
                                                   'public', 'r4t3_to_rooms',
                                                   'energcocinar1',
                                                   'parentesco3', 'area1',
                                                   'estadocivil7', 'etecho',
                                                   'paredpreb',
                                                   'adults_per_bedroom',
                                                   'paredmad', 'r4h3',
                                                   'hhsize_to_rooms', 'r4m2',
                                                   'estadocivil4', 'eviv',
                                                   'cielorazo', 'lugar6',
                                                   'techoentrepiso',
                                                   'pisomadera',
                                                   'female_per_bedroom', 'r4h1', ...])])),
                ('classifier',
                 GradientBoostingClassifier(learning_rate=0.3, max_depth=7,
                                            n_estimators=200))])

In [20]:
# Phần chạy gridsearch 

# param_grid = {
#     'classifier__n_estimators': [50, 100, 120],  
#     'classifier__learning_rate': [0.1, 0.3,0.5],  
#     'classifier__max_depth': [3, 5, 7]  
# }

# scorer = make_scorer(f1_score, average='macro')

# grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring=scorer, cv=5)
# grid_search.fit(X_train, y_train, classifier__sample_weight=weights_train)

# # best model
# best_pipeline = grid_search.best_estimator_

# # best param
# print("Best parameters ", grid_search.best_params_)


#### Train model Gradient Boosting


In [21]:
# train model với bộ tham số tốt nhất và trọng số của các nhãn
pipeline.fit(X_train, y_train,  classifier__sample_weight=weights_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('std_scaler',
                                                  StandardScaler(),
                                                  ['v2a1', 'edjefe', 'edjefa',
                                                   'meaneduc', 'age',
                                                   'rent_to_rooms',
                                                   'v2a1_to_r4t3',
                                                   'rent_per_bedroom',
                                                   'rent_to_rooms',
                                                   'v2a1_to_r4t3',
                                                   'rent_per_person']),
                                                 ('minmax_scaler',
                                                  MinMaxScaler(),
                                                  ['qmobilephone', 'noelec',
                                                   'tipovivi3', 'techootro',
                                                   'energcocinar3',
                                                   'sanitario3', '...
                                                   'public', 'r4t3_to_rooms',
                                                   'energcocinar1',
                                                   'parentesco3', 'area1',
                                                   'estadocivil7', 'etecho',
                                                   'paredpreb',
                                                   'adults_per_bedroom',
                                                   'paredmad', 'r4h3',
                                                   'hhsize_to_rooms', 'r4m2',
                                                   'estadocivil4', 'eviv',
                                                   'cielorazo', 'lugar6',
                                                   'techoentrepiso',
                                                   'pisomadera',
                                                   'female_per_bedroom', 'r4h1', ...])])),
                ('classifier',
                 GradientBoostingClassifier(learning_rate=0.3, max_depth=7,
                                            n_estimators=200))])

Ở đây, t không dùng `feature_importances` để lọc ra các feature quan trọng vì kết quả submit thấp hơn so với lúc chưa dùng.

In [22]:
# #Feature importance
# classifier = pipeline.named_steps['classifier']
# feature_importances = classifier.feature_importances_
# feature_importances = 100.0 * (feature_importances / feature_importances.max())
# sorted_idx = np.argsort(feature_importances)

# # lọc các đặc trưng đóng góp hơn 1%
# important_features_indices = np.where(feature_importances >=1 )[0]
# important_features = X_train.columns[important_features_indices]
# #  Lọc các đặc trưng quan trọng từ tập train test

# X_train_selected = X_train[important_features]
# X_test_selected = X_test[important_features]

# # Loại bỏ các đặc trưng trong pipeline
# to_dropi= np.where(feature_importances < 1 )[0]
# not_ipt = X_train.columns[to_dropi]
# col_std = [x for x in col_std if x not in not_ipt]
# col_minmax = [x for x in col_minmax if x not in not_ipt]

## Train lại model với dữ liệu mới


### Dự đoán trên dữ liệu test

In [23]:
y_pred = pipeline.predict(X_test)
best_f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score: ", best_f1)

F1 score:  0.9310433948630394


Mô hình cho ra f1-score khá tốt.

### Dự đoán với dữ liệu test để submit

In [24]:
data_test_id = list(test.Id)
data_test = test.drop(columns=['Id', 'idhogar', 'Target'])

gb_submission = pd.DataFrame({'Id': data_test_id, 'Target': pipeline.predict(data_test).astype(int)})
gb_submission.to_csv('submission.csv', index = False)